# Cálculo de Temperatura Média (Tmed)

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# -*- coding: utf-8 -*-
"""
Cálculo de Temperatura Média (Tmed) = (Tmax + Tmin)/2

Entrada:
    E:\CLIMBRA_SSP245\TMax_input\<MODELO-tasmax-ssp245>\<arquivos_tasmax.csv>
    E:\CLIMBRA_SSP245\TMin_input\<MODELO-tasmin-ssp245>\<arquivos_tasmin.csv>

Saída:
    E:\CLIMBRA_SSP245\Tmed_output\<MODELO-tasmax-ssp245>\<arquivos_Tmed.csv>

Associação:
    ACCESS-CM2-tasmax-ssp245_-25.125_-49.125.csv
       ↔ ACCESS-CM2-tasmin-ssp245_-25.125_-49.125.csv
"""

from pathlib import Path
import pandas as pd

# ==========================
# CONFIGURAÇÕES
# ==========================

TMAX_DIR = RAW_DIR / 'TMax_input'
TMIN_DIR = RAW_DIR / 'TMin_input'
TMED_DIR = RAW_DIR / 'Tmed_output'

# Nome da coluna que contém os valores
VALUE_COL = "valor"

# Sufixo do arquivo de saída
OUTPUT_SUFFIX = "_Tmed"

# ==========================
# FUNÇÕES
# ==========================

def process_temperature_files(tmax_path: Path, tmin_path: Path, base_tmax: Path, base_out: Path):
    """
    Lê um arquivo Tmax e o correspondente Tmin,
    calcula Tmed e salva na pasta equivalente.
    """
    print(f"Processando: {tmax_path}")

    # Lê Tmax
    df_max = pd.read_csv(tmax_path, sep=";")
    if VALUE_COL not in df_max.columns:
        raise KeyError(f"Coluna '{VALUE_COL}' não encontrada em {tmax_path}")

    # Lê Tmin
    df_min = pd.read_csv(tmin_path, sep=";")
    if VALUE_COL not in df_min.columns:
        raise KeyError(f"Coluna '{VALUE_COL}' não encontrada em {tmin_path}")

    # Verifica se datas batem
    for col in ["dia", "mes", "ano"]:
        if col not in df_max.columns or col not in df_min.columns:
            raise KeyError(f"Coluna de data '{col}' não encontrada em Tmax ou Tmin")
    if not df_max[["dia", "mes", "ano"]].equals(df_min[["dia", "mes", "ano"]]):
        raise ValueError(f"As datas de Tmax e Tmin não coincidem para o arquivo: {tmax_path.name}")

    # Calcula Tmed
    tmax = df_max[VALUE_COL].astype(float)
    tmin = df_min[VALUE_COL].astype(float)
    tmed = (tmax + tmin) / 2.0

    # Monta DataFrame de saída
    df_out = df_max.copy()
    df_out["Tmax"] = tmax
    df_out["Tmin"] = tmin
    df_out["Tmed"] = tmed

    # Determina pasta de saída preservando estrutura de Tmax
    rel_parent = tmax_path.relative_to(base_tmax).parent
    out_dir = base_out / rel_parent
    out_dir.mkdir(parents=True, exist_ok=True)

    # Nome do arquivo
    out_name = tmax_path.stem + OUTPUT_SUFFIX + tmax_path.suffix
    out_path = out_dir / out_name

    # Salvar
    df_out.to_csv(out_path, sep=";", index=False)
    print(f"  -> Salvo em: {out_path}")


def montar_caminho_tmin(fmax: Path, base_tmax: Path, base_tmin: Path) -> Path:
    """
    A partir do caminho do arquivo Tmax, monta o caminho do arquivo Tmin,
    trocando 'tasmax' por 'tasmin' na pasta e no nome do arquivo.
    """
    # caminho relativo do Tmax em relação à raiz TMax_input
    rel = fmax.relative_to(base_tmax)  # ex: ACCESS-CM2-tasmax-ssp245\ACCESS-CM2-tasmax-ssp245_-25.125_-49.125.csv

    # separa pasta e nome do arquivo
    rel_parent = rel.parent                      # ex: ACCESS-CM2-tasmax-ssp245
    rel_name   = rel.name                        # ex: ACCESS-CM2-tasmax-ssp245_-25.125_-49.125.csv

    # troca 'tasmax' por 'tasmin' na pasta e no nome
    rel_parent_min = Path(str(rel_parent).replace("tasmax", "tasmin"))
    rel_name_min   = rel_name.replace("tasmax", "tasmin")

    # monta caminho final em TMin_input
    fmin = base_tmin / rel_parent_min / rel_name_min
    return fmin

# ==========================
# SCRIPT PRINCIPAL
# ==========================

def main():
    base_tmax = Path(TMAX_DIR)
    base_tmin = Path(TMIN_DIR)
    base_out  = Path(TMED_DIR)
    base_out.mkdir(parents=True, exist_ok=True)

    files_tmax = sorted(base_tmax.rglob("*.csv"))

    if not files_tmax:
        print("Nenhum arquivo Tmax encontrado.")
        return

    print(f"Encontrados {len(files_tmax)} arquivos Tmax.\n")

    for i, fmax in enumerate(files_tmax, start=1):
        print(f"[{i}/{len(files_tmax)}]")

        # monta caminho do Tmin correspondente usando a regra tasmax -> tasmin
        fmin = montar_caminho_tmin(fmax, base_tmax, base_tmin)

        if not fmin.exists():
            print(f"ERRO: Arquivo Tmin correspondente não encontrado para: {fmax}")
            print(f"      Caminho esperado: {fmin}\n")
            continue

        try:
            process_temperature_files(fmax, fmin, base_tmax, base_out)
        except Exception as e:
            print(f"ERRO ao processar {fmax}: {e}\n")

if __name__ == "__main__":
    main()